In [1]:
# IMPORTS
import cvxpy as cp
import numpy as np
import numpy.linalg
import numpy.matlib
from numpy.matlib import repmat
import csv   

from sklearn.preprocessing import PolynomialFeatures

from scipy.linalg import lstsq
from scipy.integrate import odeint

import pandas as pd

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

####################### FUNCTIONS #######################

# Simulate dynamical system

def lorenz63(x, t, sigma=10, rho=28, beta=8/3):
    return [
        sigma * (x[1] - x[0]),
        x[0] * (rho - x[2]) - x[1],
        x[0] * x[1] - beta * x[2]
    ]

sig=10
rho=28
beta=8/3


####################### GENERATE DATA #######################

# Simulate dynamics

t0, tf = 0, 2.2  # start and end
dt = 0.01  # time step
n_times = int(np.floor((tf-t0)/dt)+ 1)  #Number of time instances

n_states = 3 #Number of state variables
t_span = np.linspace(t0, tf, n_times)
t = t_span

x0 = [-8, 7, 27]
X = odeint(lorenz63, x0, t_span)
x1 = X[:,0]
x2 = X[:,1]
x3 = X[:,2]

#Compute true derivatives
dX = np.zeros((len(t_span),n_states))
dX[:,0] = sig * (x2 - x1)
dX[:,1] =  x1 * (rho - x3) - x2
dX[:,2] = x1 * x2 - beta * x3

#Corrupt states by adding noise --> Observation model y(t) = x(t) + e(t)
sigma = 2
n_samples = 100

for state in range(n_states):
            
        E = sigma * np.random.randn(n_samples, n_times)#Additive zero-mean white noise (Assumed Gaussian)
        Y = repmat(X[:,state].T,n_samples,1) + E         

        filename = 'Lorenz63_samples_noise_' + str(sigma) + '_Y' + str(state+1) + '.csv'
        # writing to csv file   
        with open(filename, 'w') as csvfile:   
            # creating a csv writer object   
            csvwriter = csv.writer(csvfile)   

            # writing the data rows   
            csvwriter.writerows(Y)

dynamics = np.concatenate(([t],X.T,dX.T), axis = 0)
filename = 'Lorenz63_dynamics.csv'
# writing to csv file   
with open(filename, 'w') as csvfile:   
    # creating a csv writer object   
    csvwriter = csv.writer(csvfile)   

    # writing the data rows   
    csvwriter.writerows(dynamics)